## Introduction

Dis lesson go cover:  
- Wetin be function calling and wetin e dey use do  
- How you fit create function call wit Azure OpenAI  
- How you fit put function call inside app  

## Learning Goals

After you finish dis lesson, you go sabi how to and understand:  

- Why e good to use function calling  
- How to setup Function Call wit Azure Open AI Service  
- How to design better function calls wey go fit your app use case  


## Understand How Function Call Work

For dis lesson, we wan build one feature for our education startup wey go allow users use chatbot to find technical courses. We go recommend courses wey match dia skill level, current role, and technology wey dem dey interested in.

To finish dis work, we go use combination of:
 - `Azure Open AI` to create chat experience for di user
 - `Microsoft Learn Catalog API` to help users find courses based on wetin dem request
 - `Function Calling` to take di user's query and send am go one function wey go make di API request.

To start, make we look why we go wan use function calling in di first place:

print("Messages wey dey next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get new response from GPT wey fit see di function response


print(second_response.choices[0].message)


### Why Function Calling

If you don learn any oda lesson for dis course, you go don sabi di power wey Large Language Models (LLMs) get. We dey hope say you fit also see some of di wahala wey dem get. 

Function Calling na one feature for Azure Open AI Service wey dey help solve dis kind wahala: 
1) Make response format dey consistent 
2) Make e possible to use data from oda sources for one application inside chat context 

Before Function Calling, di response wey LLM dey give no dey structured and e no dey consistent. Developers go need write plenty complex validation code to make sure say dem fit handle di different ways wey response fit take show. 

Users no fit get answer like "Wetn be di current weather for Stockholm?". Dis na because di models dey limited to di time wey dem train di data. 

Make we look di example wey dey below wey go show dis problem: 

Make we say we wan create one database of student data so we fit suggest di correct course to dem. Below, we get two descriptions of students wey dey very similar for di data wey dem carry.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

We wan send dis one go LLM make e fit arrange di data. Later, we fit use am for our app to send am go API or keep am for database.

Make we create two prompts wey be di same wey go tell di LLM di kind information wey we dey find:


We wan send dis one to LLM make e fit check di parts wey dey important to our product. So we fit create two same prompts to tell di LLM wetin to do:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Afta we don create dis two prompts, we go send dem to di LLM by usin `openai.ChatCompletion`. We go store di prompt for di `messages` variable and give di role to `user`. Dis na to mimic message wey user dey write to chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Now we fit send both requests to di LLM and check di response we go receive.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Even though di prompts dey same and di descriptions dey similar, we fit get different formats for di `Grades` property.

If you run di cell wey dey up many times, di format fit be `3.7` or `3.7 GPA`.

Dis na because di LLM dey take unstructured data wey dey inside di written prompt and e dey return unstructured data too. We need structured format so we go sabi wetin to expect when we wan store or use di data.

If we use functional calling, we fit make sure say we go dey receive structured data back. When we dey use function calling, di LLM no dey actually call or run any functions. Instead, we go create structure wey di LLM go follow for e responses. We go then use di structured responses to sabi which function we go run for our applications.


![Function Calling Flow Diagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.pcm.png)


We fit take wetin de return from di function send am back to di LLM. Di LLM go then respond using natural language to ansa di user question.


### Use Cases for using function calls 

**Call External Tools**  
Chatbots dey good for ansa di questions wey users dey ask. Wit function calling, di chatbots fit use di message wey user send to do some kain task. For example, student fit tell chatbot say "Send email to my instructor say I need more help for dis subject". Dis one fit make di chatbot call function wey be `send_email(to: string, body: string)`.

**Make API or Database Queries**  
Users fit find information wit natural language wey go turn to formatted query or API request. Example na teacher wey fit ask "Who be di students wey finish di last assignment" wey fit make di chatbot call function wey be `get_completed(student_name: string, assignment: int, current_status: string)`.

**Create Structured Data**  
Users fit take one block of text or CSV and use di LLM to comot di important information wey dey inside. For example, student fit change one Wikipedia article about peace agreements to make AI flash cards. Dis one fit happen wit function wey be `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. How to Make Your First Function Call

To make function call, e get 3 main steps wey you go follow:  
1. Use Chat Completions API take call your list of functions plus user message.  
2. Check wetin the model respond so you fit do action like run function or API Call.  
3. Call Chat Completions API again with wetin your function respond so you fit use am create reply for the user.  


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.pcm.png)


### Elements of a function call 

#### Users Input 

Di first step na to create user message. You fit assign am dynamic by using di value wey dey text input or you fit put value for here. If na your first time wey you dey work with di Chat Completions API, we go need define di `role` and di `content` of di message. 

Di `role` fit be `system` (to set rules), `assistant` (di model) or `user` (di end-user). For function calling, we go set am as `user` and example question. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### How to create functions.

Next thing we go do na to define one function and di parameters wey dey inside di function. We go use only one function here wey dem call `search_courses` but you fit create plenty functions.

**Important**: Functions dey inside di system message to di LLM and dem go dey count inside di number of tokens wey you get available.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitions** 

`name` - Na di name of di function wey we wan make e run. 

`description` - Dis na di description of how di function dey work. For here, e dey important to make am clear and straight. 

`parameters` - List of values and format wey you wan make di model produce for im response. 

`type` - Di data type wey di properties go dey store inside. 

`properties` - List of di specific values wey di model go use for im response. 

`name` - Di name of di property wey di model go use for im formatted response. 

`type` - Di data type of dis property. 

`description` - Description of di specific property. 

**Optional**

`required` - Di property wey dem need for di function call to complete. 


### How to call di function
Afta we don define di function, di next step na to put am inside di call wey go go Chat Completion API. We go do dis one by adding `functions` to di request. For dis case, `functions=functions`.

E get option wey fit make us set `function_call` to `auto`. Dis one mean say we go allow di LLM choose which function e go call based on wetin di user message talk, instead of us choosing am by ourselves.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Make we check di response and see how dem arrange am:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

You fit see say di name of di function dey call, and from wetin di user talk, di LLM fit find di data wey go match di arguments for di function.


## 3. How to Put Function Call Inside App

Afta we don test di formatted response wey come from di LLM, na time to put am inside app.

### How to Manage di Flow

To put dis one inside our app, make we follow dis steps:

First, make we call di Open AI services and keep di message for one variable wey we go call `response_message`.


In [ ]:
response_message = response.choices[0].message

Now we go define di function wey go call di Microsoft Learn API to get list of courses:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



As e good make we do am well, we go first check if di model wan call one function. After dat, we go create one of di functions wey dey available and match am to di function wey dem dey call.  
We go then carry di arguments of di function and map dem to di arguments wey come from di LLM.  

Finally, we go add di function call message and di values wey di `search_courses` message return. Dis one go give di LLM all di information wey e need to take reply di user wit natural language.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Now we go send di updated message to di LLM so we fit get natural language response instead of API JSON formatted response.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge

Good job! To continue to dey learn about Azure Open AI Function Calling, you fit build: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Add more parameters for di function wey go fit help learners find more courses. You fit see di available API parameters for here: 
 - Create another function call wey go collect more information from di learner like dia native language 
 - Add error handling for when di function call and/or API call no return any suitable courses


<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translet service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translet. Even as we dey try make am correct, abeg sabi say AI translet fit get mistake or no dey accurate well. Di original dokyument for im native language na di one wey you go take as di correct source. For important mata, e good make professional human translet am. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translet.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
